In [19]:
import json
import pandas as pd

In [20]:
with open("../data/places.json") as places_file:
    places_dict = json.load(places_file)

In [21]:
df_places = pd.json_normalize(places_dict, record_path="features")
df_places

,type,properties.identifier,properties.name,properties.information,properties.categories,properties.campus,properties.faculties,properties.floors,properties.needApproval,geometry.type,geometry.coordinates
0,Feature,auditorium-33669891,Auditorio Sergio Larraín,,[auditorium],LC,,[1],False,Point,"[-70.61727595494885, -33.418740654946745]"
1,Feature,auditorium-33669872,Auditorio Diseño,,[auditorium],LC,,[1],False,Point,"[-70.61730781409395, -33.41929369752461]"
2,Feature,auditorium-33079671,Auditorios FADEU,,[auditorium],LC,,[-1],False,Point,"[-70.61844935640693, -33.419592118926495]"
3,Feature,parking-19,Acceso/Salida (UC Christus) - M. Carlos Cas. 547,,[parking],SJ,,[1],False,Point,"[-70.61561067381189, -33.496231537274696]"
4,Feature,parking-18,Salida 2 - M. Carlos Cas.,,[parking],SJ,,[1],False,Point,"[-70.60817073136938, -33.49566896605042]"
...,...,...,...,...,...,...,...,...,...,...,...
1028,Feature,D,College,,[faculty],SJ,NaN,NaN,False,Polygon,"[[[-70.611342, -33.500313], [-70.610906, -33.5..."
1029,Feature,CS,Ciencias de la Salud,,[faculty],SJ,NaN,NaN,False,Polygon,"[[[-70.61044, -33.500247], [-70.609931, -33.50..."
1030,Feature,FIL,Instituto de Filosofía,,[faculty],SJ,NaN,NaN,False,Polygon,"[[[-70.614209, -33.496663], [-70.613913, -33.4..."
1031,Feature,BIO,Facultad de Ciencias Biológicas,,[faculty],CC,NaN,NaN,False,Polygon,"[[[-70.64004, -33.441616], [-70.639824, -33.44..."


In [22]:
df_places[df_places["properties.campus"].isna()] # Verificar si hay algun campus null

,type,properties.identifier,properties.name,properties.information,properties.categories,properties.campus,properties.faculties,properties.floors,properties.needApproval,geometry.type,geometry.coordinates


In [23]:
df_places["properties.categories"]

0       [auditorium]
1       [auditorium]
2       [auditorium]
3          [parking]
4          [parking]
            ...     
1028       [faculty]
1029       [faculty]
1030       [faculty]
1031       [faculty]
1032      [building]
Name: properties.categories, Length: 1033, dtype: object

In [25]:
all_categories = set()
for i in df_places["properties.categories"]:
    for c in i:
        all_categories.add(c)
all_categories

{'',
 'auditorium',
 'bath',
 'building',
 'classroom',
 'computers',
 'faculty',
 'financial',
 'food_lunch',
 'laboratory',
 'other',
 'park_bicycle',
 'parking',
 'photocopy',
 'shop',
 'sports_place',
 'studyroom',
 'trash',
 'water'}

In [18]:
df_places[df_places["properties.categories"] == [None]]

ValueError: ('Lengths must match to compare', (1033,), (1,))